In [ ]:
from dasilva_invariants import meshes, invariants
import glob
import numpy as np
import pylab as plt
import gc
import progressbar
%matplotlib inline

In [ ]:
files = glob.glob('/glade/u/home/danieldas/scratch/data/Advection/March2015-1s/*mhd*.hdf')
files.sort()
files = files[::60*5]
files[:10]

In [ ]:
local_times = np.arange(0, 360, 15, dtype=float)
radii = np.arange(3, 8.5, 0.5, dtype=float)

print('Local times', local_times.size)
print('Radii', radii.size)

In [ ]:
params = []

for file in files:
    for local_time in local_times:
        for radius in radii:
            phi = np.deg2rad(local_time)
            x = radius * np.cos(phi)
            y = radius * np.sin(phi)
            params.append((file, local_time, radius, (x, y, 0)))

In [ ]:
LStar = {}
bar = progressbar.ProgressBar()

for file, local_time, radius, starting_point in bar(params):
    mesh = meshes.get_dipole_mesh_on_lfm_grid(file)

    LStar[file, local_time, radius] = invariants.calculate_LStar(mesh, starting_point, 0).LStar

    del mesh
    gc.collect()

In [ ]:
tstamps = [f.split('T')[1].replace('-', '')[:4] + 'UT' for f in files]
tstamps[2:10]

In [ ]:
s = ''

for f, tstamp in zip(files[2:], tstamps[2:]):
    s += tstamp + '\n'
    for local_time in local_times:
        for radius in radii:
            s += '%.14f' % LStar[f, local_time, radius] + '\t'
        s += '\n'

#print(s)

fh = open('zhao_grid.txt', 'w')
fh.write(s)
fh.close()

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

for f, tstamp in zip(files[2:], tstamps[2:]):
    arr = np.zeros((local_times.size, radii.size))
    for i, local_time in enumerate(local_times):
        for j, radius in enumerate(radii):
            arr[i, j] = LStar[f, local_time, radius]

    fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
    im = ax.pcolor(np.deg2rad(local_times), radii, arr.T, vmin=2, vmax=10)
    ax.set_title(tstamp)
    